# सेम्यान्टिक कर्नेल उपकरण प्रयोग उदाहरण

यो दस्तावेजले सेम्यान्टिक कर्नेलमा आधारित उपकरण निर्माण गर्न प्रयोग गरिएको कोडको अवलोकन र व्याख्या प्रदान गर्दछ, जसले Azure AI Search सँग Retrieval-Augmented Generation (RAG) को लागि एकीकृत गर्दछ। यो उदाहरणले कसरी एक AI एजेन्ट निर्माण गर्ने देखाउँछ, जसले Azure AI Search इन्डेक्सबाट यात्रा सम्बन्धी दस्तावेजहरू प्राप्त गर्दछ, प्रयोगकर्ताको प्रश्नहरूलाई सेम्यान्टिक खोज परिणामहरूसँग समृद्ध बनाउँछ, र विस्तृत यात्रा सिफारिसहरू स्ट्रिम गर्दछ।


### प्याकेजहरू आयात गर्दै
तलको कोडले आवश्यक प्याकेजहरू आयात गर्दछ:


In [ ]:
import json
import os

from typing import Annotated

from IPython.display import display, HTML

from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

### सेम्यान्टिक कर्नेल र एआई सेवा सिर्जना गर्दै

सेम्यान्टिक कर्नेलको एउटा उदाहरण असिन्क्रोनस OpenAI च्याट कम्प्लिसन सेवासँग सिर्जना र कन्फिगर गरिन्छ। यो सेवा प्रतिक्रियाहरू उत्पन्न गर्न प्रयोग गर्नका लागि कर्नेलमा थपिन्छ।


In [ ]:
load_dotenv()
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### प्रम्प्ट प्लगइन परिभाषित गर्दै

PromptPlugin एउटा देशीय प्लगइन हो जसले पुनःप्राप्त सन्दर्भ प्रयोग गरेर परिमार्जित प्रम्प्ट निर्माण गर्नका लागि एउटा कार्य परिभाषित गर्दछ।


In [ ]:
class SearchPlugin:

    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context or function results.",
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "First review the retrieved context, if this does not answer the query, try calling an available plugin functions that might give you an answer. If no context is available, say so."
        )
    
    @kernel_function(
        name="retrieve_documents",
        description="Retrieve documents from the Azure Search service.",
    )
    def get_retrieval_context(self, query: str) -> str:
        results = self.search_client.search(query)
        context_strings = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "\n\n".join(context_strings) if context_strings else "No results found"

In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

## भेक्टर डाटाबेस सुरुवात

हामी Azure AI Search लाई स्थायी भण्डारणसहित सुरु गर्छौं र उन्नत नमूना कागजातहरू थप्छौं। Azure AI Search कागजातहरू भण्डारण र पुनःप्राप्त गर्न प्रयोग गरिनेछ, जसले सही उत्तरहरू उत्पन्न गर्न सन्दर्भ प्रदान गर्छ।


In [ ]:
# Initialize Azure AI Search with persistent storage
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "travel-documents"

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Define the index schema
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Check if index already exists if not, create it
try:
    existing_index = index_client.get_index(index_name)
    print(f"Index '{index_name}' already exists, using the existing index.")
except Exception:
    # Create the index if it doesn't exist
    print(f"Creating new index '{index_name}'...")
    index_client.create_index(index)


# Enhanced sample documents
documents = [
    {"id": "1", "content": "Contoso Travel offers luxury vacation packages to exotic destinations worldwide."},
    {"id": "2", "content": "Our premium travel services include personalized itinerary planning and 24/7 concierge support."},
    {"id": "3", "content": "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage."},
    {"id": "4", "content": "Popular destinations include the Maldives, Swiss Alps, and African safaris."},
    {"id": "5", "content": "Contoso Travel provides exclusive access to boutique hotels and private guided tours."}
]

# Add documents to the index
search_client.upload_documents(documents)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[SearchPlugin(search_client=search_client), WeatherInfoPlugin()],
    name="TravelAgent",
    instructions="Answer travel queries using the provided tools and context. If context is provided, do not say 'I have no context for that.'",
)

### स्ट्रिमिङ आह्वानसँग एजेन्ट चलाउने

मुख्य असिन्क्रोनस लूपले संवादका लागि एउटा थ्रेड सिर्जना गर्छ, र प्रत्येक प्रयोगकर्ता इनपुटका लागि, ताकि एजेन्टले पुनःप्राप्ति सन्दर्भ देख्न सकोस्। प्रयोगकर्ताको सन्देश पनि थपिन्छ, र त्यसपछि एजेन्टलाई स्ट्रिमिङ प्रयोग गरेर आह्वान गरिन्छ। उत्पादनलाई स्ट्रिमिङ हुँदै गर्दा प्रिन्ट गरिन्छ।


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


# शीर्षक

यो फाइलले Markdown को आधारभूत संरचना प्रदर्शन गर्दछ। Markdown एक हल्का मार्कअप भाषा हो जसले HTML जस्तै जटिल संरचना बिना नै सामग्रीलाई सजिलै ढाँचाबद्ध गर्न अनुमति दिन्छ।

## खण्ड १: परिचय

Markdown प्रयोग गर्न सजिलो छ र यसले लेखनलाई सरल बनाउँछ। यो विशेष गरी प्रोग्रामर र लेखकहरूका लागि उपयोगी छ। Markdown को केही मुख्य फाइदाहरू:

- पठनीयता: Markdown फाइलहरू पढ्न र सम्पादन गर्न सजिलो हुन्छ।
- लचिलोपन: Markdown विभिन्न प्लेटफर्महरूमा समर्थन गर्दछ।
- सरलीकरण: HTML को तुलनामा Markdown कम जटिल छ।

[!NOTE] Markdown को प्रयोगले तपाईंको कामलाई व्यवस्थित र प्रभावकारी बनाउन सक्छ।

## खण्ड २: Markdown को आधारभूत तत्वहरू

Markdown मा विभिन्न प्रकारका तत्वहरू छन्। यहाँ केही सामान्य तत्वहरूको सूची छ:

### शीर्षकहरू

Markdown मा शीर्षकहरू `#` प्रतीक प्रयोग गरेर बनाइन्छ। शीर्षकको स्तर `#` को संख्याले निर्धारण गर्दछ। उदाहरण:

```text
User: 
Can you explain Contoso's travel insurance coverage?

Function Calls (click to expand)

Calling function: retrieve_documents({"query": "Contoso travel insurance coverage"})

Function Result:

Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.

Document: Contoso Travel provides exclusive access to boutique hotels and private guided tours.

Document: Our premium travel services include personalized itinerary planning and 24/7 concierge support.

TravelAgent:

Contoso's travel insurance coverage includes the following:

1. **Medical Emergencies**: Coverage for unforeseen medical issues that may arise while traveling.
2. **Trip Cancellations**: Protection in case you need to cancel your trip for covered reasons.
3. **Lost Baggage**: Compensation for baggage that is lost during your trip.

If you need more specific details about the policy, it would be best to contact Contoso directly or refer to their official documentation.
```

### सूचीहरू

Markdown ले क्रमबद्ध र अक्रमबद्ध सूचीहरू समर्थन गर्दछ। उदाहरण:

#### अक्रमबद्ध सूची:
- वस्तु १
- वस्तु २
- वस्तु ३

#### क्रमबद्ध सूची:
1. चरण १
2. चरण २
3. चरण ३

### लिङ्कहरू

Markdown मा लिङ्कहरू बनाउन सजिलो छ। उदाहरण:

@@INLINE_CODE_0@@

### कोड ब्लकहरू

Markdown ले कोड ब्लकहरूलाई समर्थन गर्दछ। उदाहरण:

@@CODE_BLOCK_1@@

[!TIP] कोड ब्लकहरू प्रयोग गर्दा, तपाईंले आफ्नो कोडलाई स्पष्ट र व्यवस्थित बनाउन सक्नुहुन्छ।

## खण्ड ३: Markdown को उपयोगिता

Markdown विभिन्न प्रकारका परियोजनाहरूमा उपयोगी छ। उदाहरणका लागि:

- प्रलेखन लेखन
- ब्लग पोस्टहरू
- नोटहरू लिने
- प्रोजेक्ट README फाइलहरू

[!IMPORTANT] Markdown को सही प्रयोगले तपाईंको सामग्रीलाई अझ प्रभावकारी बनाउँछ।

## खण्ड ४: निष्कर्ष

Markdown एक शक्तिशाली तर सरल उपकरण हो। यसको प्रयोगले तपाईंलाई लेखन र ढाँचाबद्ध गर्न समय बचत गर्न मद्दत गर्दछ। Markdown सिक्न र प्रयोग गर्न सजिलो छ, जसले गर्दा यो सबैका लागि उपयुक्त हुन्छ। Markdown को प्रयोग गरेर तपाईं आफ्नो सामग्रीलाई व्यवस्थित र आकर्षक बनाउन सक्नुहुन्छ।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धि हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।  
